# Binary star evolution and binary black holes

All the relevant information for the project are to be found in the pdf document present in the repo.
Note that you are assigned to project 1 (as the title said).

## Datasets 

Datasets are stored on Google Drive (link and description in the pdf document)

### Contacts

* Michela Mapelli <michela.mapelli@unipd.it>


In [ ]:
import pandas as pd
import numpy as np
import string
import glob
import os
import seaborn as sns
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

**Ognuno di voi dovrebbe provare a runnare os.getlogin() per vedere cosa esce, poi metterlo nell'if e aggiornare il path di conseguenza, così eliminiamo il lavoro di cambio di directory ogni volta. (Ho provato a fare un guess del path di Tommaso).**

In [ ]:
# personalized computer settings

hname = os.getlogin()
path = ''

if hname=='paolozinesi': path = '/Users/paolozinesi/Downloads/Project_LCP_A/stable_MT_vs_CE/'
if hname=='tommaso': path = "/Users/tommaso/Desktop/MagistraleI/TheFormationOfBinaryBlackHoles_data/lab_data/stable_MT_vs_CE/"
#if hname=='NICOLAZOMER': path = "PATHDINICOLA"
#if hname=='ANDREALAZZARI': path = "PATHDIANDREA"

In [ ]:
alpha_values = [0.5, 1, 3, 5]
met_values = [0.02, 0.002, 0.0002, 0.004, 0.0004, 0.006, 0.008, 0.012, 0.0012, 0.016, 0.0016]

frame = pd.DataFrame(data=None, columns=['col.0:ID','col.1:m1ZAMS/Msun', 'col.2:m2ZAMS/Msun', 'col.3:m1rem/Msun','col.4:m2rem/Msun',  'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.21:CE'])

for alpha in alpha_values:
    for met in met_values:
        for name in glob.glob(path+f'A{alpha}/MTCE_BBHs_{met}*'):
        #for name in glob.glob(f'/Users/tommaso/Desktop/MagistraleI/TheFormationOfBinaryBlackHoles_data/lab_data/stable_MT_vs_CE/A{alpha}/MTCE_BBHs_{met}*'):    
        #for name in glob.glob(f'/data/TheFormationOfBinaryBlackHoles_data/stable_MT_vs_CE/A{alpha}/MTCE_BBHs_{met}*'):
            df = pd.read_csv(name, skiprows=2, header=0, sep=' ')
            df = df.loc[:,['col.0:ID','col.1:m1ZAMS/Msun', 'col.2:m2ZAMS/Msun','col.3:m1rem/Msun','col.4:m2rem/Msun',  'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.21:CE']]
            met_array = np.ones((df.shape[0], 1))*met
            alpha_array = np.ones((df.shape[0],1))*alpha
            df['metallicity'] = met_array
            df['alpha'] = alpha_array
            
            frame = pd.concat([frame, df], axis=0, ignore_index=True)


In [ ]:
frame.head()

In [ ]:
#Creating the a dataframe with common envelopes and one with mass transfer
frame_true = frame[(frame['col.21:CE'] == True)] 
frame_true = frame_true.reset_index(drop=True)
frame_false = frame[(frame['col.21:CE'] == False)] 
frame_false = frame_false.reset_index(drop=True)

In [ ]:
frame_true.head()



In [ ]:
frame_false.head()

In [ ]:
#frame_graph = frame.drop(columns=['col.0:ID'], axis=1)
#sns.pairplot(frame, hue = 'col.21:CE', palette='Set2')

In [ ]:
#frame_graph.head()

**!!! WARNING, il blocco sotto ci mette un paio d'ore per runnare**

In [ ]:



#frame_graph = frame.drop(columns=['col.0:ID', 'metallicity', 'alpha'], axis=1)
#sns.pairplot(frame_graph, hue = 'col.21:CE', palette='Set2')
                    

## ML algorithms

In [ ]:
m_train = 20000
m_test = 1000

frame_sample = pd.concat([frame_true.sample(n=int(0.5*(m_train+m_test))),
                          frame_false.sample(n=int(0.5*(m_train+m_test)))], ignore_index=True)

frame_train = frame_sample.iloc[:m_train]
frame_test = frame_sample.iloc[-m_test:]


# convert dataframe into a numpy array
X_train = frame_train.drop(labels=["col.0:ID","col.21:CE"], axis=1).values
Y_train = frame_train["col.21:CE"].apply(lambda x: np.sum(x)).values
X_test = frame_test.drop(labels=["col.0:ID","col.21:CE"], axis=1).values
Y_test = frame_test["col.21:CE"].apply(lambda x: np.sum(x)).values

#### Random Forest

In [ ]:
RF_clf = RandomForestClassifier(max_depth=None)
RF_clf.fit(X_train, Y_train)

train_err = 1 - RF_clf.score(X_train, Y_train)
print("Training error = %1.3f" % train_err)

test_err = 1 - RF_clf.score(X_test, Y_test)
print("Test error = %1.3f" % test_err)

#### SVM

In [ ]:
SVM_clf = sklearn.svm.SVC()
SVM_clf.fit(X_train, Y_train)

train_err = 1 - SVM_clf.score(X_train, Y_train)
print("Training error = %1.3f" % train_err)

test_err = 1 - SVM_clf.score(X_test, Y_test)
print("Test error = %1.3f" % test_err)

#### Neural Network

In [ ]:
NN_clf = MLPClassifier(hidden_layer_sizes=(50,), learning_rate='adaptive')
NN_clf.fit(X_train, Y_train)

train_err = 1 - NN_clf.score(X_train, Y_train)
print("Training error = %1.3f" % train_err)

test_err = 1 - NN_clf.score(X_test, Y_test)
print("Test error = %1.3f" % test_err)

In [ ]:
frame_sample.head()
frame_sample_2 = frame_sample.drop(columns=['col.0:ID'])
frame_sample_2.head()

In [ ]:
sns.pairplot(frame_sample_2, hue = 'col.21:CE', palette='Set2')    

In [ ]:
plt.hist(frame_sample_2[frame_sample_2['col.21:CE']==True]['metallicity'], bins = 50, alpha = 0.5)
plt.hist(frame_sample_2[frame_sample_2['col.21:CE']==False]['metallicity'], bins = 50, alpha = 0.5)
plt.show()

In [ ]:
frame_false

In [ ]:
plt.hist(frame_true['metallicity'], bins = 40, alpha = 0.5, density=True, color='r')
plt.hist(frame_false['metallicity'], bins = 40, alpha = 0.5, density=True, color='g')
plt.yscale('log')
plt.show()

In [ ]:
frame_true[frame_true['metallicity']==0.008].count()

In [ ]:
frame_false[frame_false['metallicity']==0.02].count()

In [ ]:
frame_false['metallicity'].unique()

In [ ]:

fig, ax = plt.subplots(nrows= 2, ncols=2, figsize=(20, 7))
ax[0,0].hist(frame_true['col.1:m1ZAMS/Msun'], bins='fd', density=True, alpha=0.5, label='True')
ax[0,0].hist(frame_false['col.1:m1ZAMS/Msun'], bins='fd', density=True, alpha=0.5, label='False')
ax[0,1].hist(frame_true['col.3:m1rem/Msun'], bins='fd', density=True, alpha=0.5, label='True')
ax[0,1].hist(frame_false['col.3:m1rem/Msun'], bins='fd', density=True, alpha=0.5, label='False')
ax[1,0].hist(frame_true['col.4:m2rem/Msun'], bins='fd', density=True, alpha=0.5, label='True')
ax[1,0].hist(frame_false['col.4:m2rem/Msun'], bins='fd', density=True, alpha=0.5, label='False')
ax[1,1].hist(frame_true['col.2:m2ZAMS/Msun'], bins='fd', density=True, alpha=0.5, label='True')
ax[1,1].hist(frame_false['col.2:m2ZAMS/Msun'], bins='fd', density=True, alpha=0.5, label='False')


fig.legend()
fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols=1, figsize=(7, 7))
ax.hist(frame_true['col.7:sma/Rsun'], bins='fd', density=True, alpha=0.5, label='True')
ax.hist(frame_false['col.7:sma/Rsun'], bins='fd', density=True, alpha=0.5, label='False')